In [1]:
!jupyter trust "Segmenting and Clustering Neighborhoods in Toronto 3.ipynb"
!jupyter nbconvert --to html "Segmenting and Clustering Neighborhoods in Toronto 3.ipynb"

Notebook already signed: Segmenting and Clustering Neighborhoods in Toronto 3.ipynb


[NbConvertApp] Converting notebook Segmenting and Clustering Neighborhoods in Toronto 3.ipynb to html
[NbConvertApp] Writing 1058269 bytes to Segmenting and Clustering Neighborhoods in Toronto 3.html


In [2]:
#!pip install beautifulsoup4
#!pip install geocoder

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import folium # map rendering library

# Segmenting and Clustering Neighborhoods in Toronto

## Instructions

### For this assignment, you will be required to explore and cluster the neighborhoods in Toronto.

Start by creating a new Notebook for this assignment.
Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas  dataframe 


In [4]:
website_text = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_text,'xml')
table = soup.find('table',{'class':'wikitable sortable'})

In [5]:
table_rows = table.find_all('tr')
data = []
for row in table_rows:
    td=[]
    for t in row.find_all('td'):
        td.append(t.text.strip())
    data.append(td)
df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighborhood'])

In [6]:
df = df[~df['Borough'].isnull()]  # to filter out bad rows
df.drop(df[df.Borough == 'Not assigned'].index, inplace=True)
df.reset_index(drop=True, inplace=True)
df = df.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ','.join(x)).reset_index()
df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


In [7]:
geo=pd.read_csv("data/Geospatial_Coordinates.csv")
geo
df=df.join(geo, lsuffix='PostalCode', rsuffix='Postal Code')
df.drop('Postal Code',axis='columns', inplace=True)
df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [8]:
df.shape

(103, 5)

## Show all neighbourhood

In [9]:
latitude=43.676357
longitude=-79.293031

map_all = folium.Map(location=[latitude, longitude], zoom_start=10)


# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], 
                                           df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_all)  
    
map_all

# Show Only Borough contain Toronto

In [10]:
torronto=df[df["Borough"].str.contains("Toronto")]
torronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [11]:
latitude=43.676357
longitude=-79.293031

map_torronto = folium.Map(location=[latitude, longitude], zoom_start=11)


# add markers to map
for lat, lng, borough, neighborhood in zip(torronto['Latitude'], torronto['Longitude'], 
                                           torronto['Borough'], torronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#000000',
        fill=True,
        fill_color='#2F32FF',
        fill_opacity=0.7,
        parse_html=False).add_to(map_torronto)  
    
map_torronto

In [12]:
pd.unique(df['Borough'])

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

In [13]:
devisor = len(pd.unique(df['Borough']))
partition=(int (int("FFFF", 16)/devisor))
start=int("1000",16)
for i in range(devisor):
    print (hex(start).replace("0x","#"))
    start=start+partition
    




#1000
#2999
#4332
#5ccb
#7664
#8ffd
#a996
#c32f
#dcc8
#f661


In [14]:
latitude=43.676357
longitude=-79.293031

map_cluster = folium.Map(location=[latitude, longitude], zoom_start=11)


# add markers to map
for lat, lng, borough, neighborhood in zip(torronto['Latitude'], torronto['Longitude'], 
                                           torronto['Borough'], torronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#000000',
        fill=True,
        fill_color='#2F32FF',
        fill_opacity=0.7,
        parse_html=False).add_to(map_cluster)  
    
map_cluster

# Random Clustering Color Based 
## Color code all Borough

In [15]:
latitude=43.676357
longitude=-79.293031

map_cluster = folium.Map(location=[latitude, longitude], zoom_start=11)

devisor = len(pd.unique(df['Borough']))
partition=(int (int("FFFF", 16)/devisor))
start=int("1000",16)


for burr in pd.unique(df['Borough']):
    #print(burr)
    temp=df[df["Borough"].str.contains(burr)]
    #print (hex(start).replace("0x","#"))
    color_map= (hex(start).replace("0x","#"))
    start=start+partition
    #print(temp.head(1))
    for lat, lng, borough, neighborhood in zip(temp['Latitude'], temp['Longitude'],
                                               temp['Borough'], temp['Neighborhood']):
        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker([lat, lng],
                            radius=5,
                            popup=label,
                            color=color_map,
                            fill=True,
                            fill_color=color_map,
                            fill_opacity=0.7,
                            parse_html=False).add_to(map_cluster)  
map_cluster